In [1]:
import cv2
import sys
def variance_of_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

In [2]:
import glob
def blurimagedetection():
    paths=glob.glob("C:/1-GG/CAP4/EventDetection/Dataset/VDO1/VDOFrames/*.jpg")
    print(len(paths))

    thvalue = 300
    #imagePath = "C:\1-GG\CAP4\EventDetection\Dataset\VDO1\VDOFrames"
    for image in paths:
        image = cv2.imread(image)
        print(image)
    # Convert to grey scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = variance_of_laplacian(gray)
    text = "Not Blurry"
    if fm < thvalue:
        text = "Blurry"	
        print('Blurry')
   # write to blurimage folder
    if fm > thvalue:
        text = "Not Blurry"	
        print('Not Blurry')

In [3]:
# Let's check image pairs with similarity larget than threshold.
# You can lower threshold to find more duplicates (and more false positives).

import matplotlib.pyplot as plt
import torch
def duplicateImage():
    hashes=duplicateHashing()
    hashes = torch.Tensor(np.array(hashes).astype(int)).cuda()
    # calc similarity scores
    sims = np.array([(hashes[i] == hashes).sum(dim=1).cpu().numpy()/256 for i in range(hashes.shape[0])])
    threshold = 0.96
    duplicates = np.where(sims > threshold)
    pairs = {}
    for i,j in zip(*duplicates):
        if i == j:
            continue
            path1 = paths[i]
            path2 = paths[j]
            print(path1)
            print(path2)
            image1 = cv2.imread(path1)
            image2 = cv2.imread(path2)
            if image1.shape[0] > image1.shape[1] / 2:
                fig,ax = plt.subplots(figsize=(20,20), ncols=2)
            elif image1.shape[1] > image1.shape[0] / 2:
                    fig,ax = plt.subplots(figsize=(20,20), nrows=2)
            else:
                        fig,ax = plt.subplots(figsize=(20,30), nrows=2)
                        ax[0].imshow(image1)
                        ax[1].imshow(image2)
                        plt.show()

In [4]:
import cv2
#pip install imagehash
import imagehash
from tqdm import tqdm_notebook as tqdm
from PIL import Image

import glob
def duplicateHashing():
    paths=glob.glob("C:/1-GG/CAP4/EventDetection/1-ExampleSetting/VDOFrames/*.jpg")
    funcs = [
    imagehash.average_hash,
    imagehash.phash,
    imagehash.dhash,
    imagehash.whash,
    #lambda x: imagehash.whash(x, mode='db4'),
    ]
    hashes = []
    for path in tqdm(paths, total=len(paths)):
        image = cv2.imread(path)
        image = Image.fromarray(image)
        hashes.append(np.array([f(image).hash for f in funcs]).reshape(256))
    return hashes    